In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from tqdm import tqdm
import os
#from plotly import plotly
#import plotly.offline as offline
#import plotly.graph_objs as go
#offline.init_notebook_mode()
from collections import Counter

In [2]:
#project_data = pd.read_csv(r'C:\Users\sreddy\OneDrive - MerckGroup\New folder/application1.log',sep=' ')


In [3]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns

In [4]:
lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName}%{GREEDYDATA:logdata}', pr)

In [5]:
print(lg.grok('''2020-03-12 20:09:02,847 [main] INFO  org.apache.spark.executor.CoarseGrainedExecutorBackend  - Started daemon with process name: 70733@deda1x3311'''))

{'timestmp': '2020-03-12 20:09:02,847', 'prg': '[main]', 'level': 'INFO', 'ClassName': 'org.apache.spark.executor.CoarseGrainedExecutorBackend', 'logdata': '  - Started daemon with process name: 70733@deda1x3311'}


In [6]:
lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName}%{GREEDYDATA:logdata}', pr)
print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0)'''))

{'timestmp': '2020-03-12 20:09:17,951', 'prg': '[Executor task launch worker for task 0]', 'level': 'ERROR', 'ClassName': 'org.apache.spark.executor.Executor', 'logdata': '  - Exception in task 0.0 in stage 0.0 (TID 0)'}


In [7]:
[%{CUSTOM_PROCESS_NAME:processName}\]%{SPACE}%{JAVACLASS:class}%{SPACE}-%{SPACE}%{JAVALOGMESSAGE:logmessage}

SyntaxError: invalid syntax (<ipython-input-7-5bd1b30ef165>, line 1)

In [8]:
lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0)
java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

{'timestmp': '2020-03-12 20:09:17,951', 'prg': '[Executor task launch worker for task 0]', 'level': 'ERROR', 'ClassName': 'org.apache.spark.executor.Executor', 'JavaMessage': ' - Exception in task 0.0 in stage 0.0 (TID', 'logdata': '0)'}


In [9]:
filename = "application1.log"

lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0)
#java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

with open(filename, 'r') as filehandle:
    for line in filehandle:
        #print(line)
        print(lg.grok(line))
        If 


None


NameError: name 'If' is not defined

In [10]:
lg = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

print(lg.grok('''java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

{'ClassName': 'java.lang.RuntimeException:', 'ClassName2': 'java.lang.NullPointerException:', 'JavaMessage': 'hive.llap.daemon.service.hosts must be', 'logdata': 'defined'}


In [11]:
lg = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

print(lg.grok('''at com.hortonworks.spark.sql.hive.llap.HiveWarehouseDataReaderFactory.createDataReader(HiveWarehouseDataReaderFactory.java:66)'''))

{'JavaStackTrace': 'at com.hortonworks.spark.sql.hive.llap.HiveWarehouseDataReaderFactory.createDataReader(HiveWarehouseDataReaderFactory.java:66)', 'class': 'com.hortonworks.spark.sql.hive.llap.HiveWarehouseDataReaderFactory', 'method': 'createDataReader', 'file': 'HiveWarehouseDataReaderFactory.java', 'line': '66', 'logdata': ''}


In [12]:
#%{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} 
lg = LineGrokker('%{JAVALOGMESSAGE:ClassMessage}%{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

a=lg.grok('''2020-01-07 13:06:03,146 [Driver] INFO  org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef  - Registered StateStoreCoordinator endpoint''')
print(a)

{'ClassMessage': '2020-01-07 13:06:03,146 [Driver] INFO  org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef  - Registered StateStoreCoordinator endpoint', 'logdata': ''}


In [13]:
with open(filename, 'r') as filehandle:
    for line in filehandle:
        #print(lg.grok(line))
        if pline=='':
            cline=lg.grok(line)
            if cline !=None
                pline=cline            
        elseif pline['level']=='ERROR':
            cline=lg2.grok(line)
            if cline !=None
                pline={**pline, **cline}
            cline=lg3.grok(line)
            if cline !=None
                pline={**pline, **cline}
            cline=lg4.grok(line)
            if cline !=None
                pline={**pline, **cline}

SyntaxError: invalid syntax (<ipython-input-13-09ee9a29de93>, line 6)

In [16]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
filename = "testlog.txt"

outF = open("testoutput", "w")

lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{JAVALOGMESSAGE:ClassMessage}%{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0)
#java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))
pline=[]
cline={}

with open(filename, 'r') as filehandle:
    for line in filehandle:
        #print(lg.grok(line))
        cline=lg.grok(line)
        if (cline is not None) and len(pline)==0:
            print(cline)
#            outF.write(str(cline))
#            outF.write("\n")
            #pline=[]
            print(1)
            if cline['level']=='ERROR':
                pline.append(cline)
            else:
                outF.write(str(cline))
                outF.write("\n")
            continue
        elif (cline is None) and len(pline)!=0:
            cline=lg2.grok(line)
            if cline is not None:
                #print(cline)
                #print(2)
                pline.append(cline)
                continue
            cline=lg3.grok(line)
            if cline is not None:
                #print(cline)
                #print(3)
                pline.append(cline)                
#            cline=lg4.grok(line)
#            if cline is not None:
#                print(cline)
#                print(4)
#                pline.append(cline)
        elif (cline is not None) and len(pline)!=0:        
            outF.write(str(pline))
            outF.write("\n")
            outF.write(str(cline))
            outF.write("\n")
            pline=[]
#        print(pline)
        
outF.close()

{'timestmp': '2020-03-12 20:09:17,951', 'prg': '[Executor task launch worker for task 0]', 'level': 'ERROR', 'ClassName': 'org.apache.spark.executor.Executor', 'JavaMessage': ' - Exception in task 0.0 in stage 0.0 (TID', 'logdata': '0)'}
1


In [25]:
import csv
#outcsv = open("testoutput.csv", "w")
with open('testoutput', 'r') as f:
    for line in f:
        a=dict(line)
        print(type(a))
#        for key in line.keys():
#              print(line[key])
#        outcsv.write("%s,%s\n"%(key,line[key]))
#outcsv.close()

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [103]:
filename = "application2.log"
#lg = LineGrokker('%{LOGLEVEL:level}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{JAVALOGMESSAGE:ClassMessage}%{GREEDYDATA:logdata}', pr)
import csv
with open(filename, 'r') as filehandle:
    for line in filehandle:
        cline=lg4.grok(line)
        print(cline)

{'ClassMessage': 'Container: container_e151_1580556634479_18136_02_000003 on awdex01126.merckgroup.com_45454_1581509898363', 'logdata': ''}
{'ClassMessage': '========================================================================================================', 'logdata': ''}
{'ClassMessage': 'LogType:directory.info', 'logdata': ''}
{'ClassMessage': 'Log Upload Time:Wed Feb 12 13:18:18 +0100 2020', 'logdata': ''}
{'ClassMessage': 'LogLength:14170', 'logdata': ''}
{'ClassMessage': 'Log Contents:', 'logdata': ''}
{'ClassMessage': 'ls -l:', 'logdata': ''}
{'ClassMessage': 'total 152', 'logdata': ''}
{'ClassMessage': 'lrwxrwxrwx 1 s112788 hadoop    78 Feb 12 13:17 __app__.jar -> /data5/hadoop/yarn/local/usercache/s112788/filecache/134/qd_rdq_2.10-1.0.1.jar', 'logdata': ''}
{'ClassMessage': 'lrwxrwxrwx 1 s112788 hadoop    62 Feb 12 13:17 __spark__.jar -> /data2/hadoop/yarn/local/filecache/4063/spark-hdp-assembly.jar', 'logdata': ''}
{'ClassMessage': 'lrwxrwxrwx 1 s112788 hadoop    94 Feb

{'ClassMessage': 'hadoop_shell_errorcode=$?', 'logdata': ''}
{'ClassMessage': 'if [ $hadoop_shell_errorcode -ne 0 ]', 'logdata': ''}
{'ClassMessage': 'then', 'logdata': ''}
{'ClassMessage': '  exit $hadoop_shell_errorcode', 'logdata': ''}
{'ClassMessage': 'fi', 'logdata': ''}
{'ClassMessage': 'ln -sf "/data4/hadoop/yarn/local/filecache/423/datanucleus-core-3.2.10.jar" "datanucleus-core-3.2.10.jar"', 'logdata': ''}
{'ClassMessage': 'hadoop_shell_errorcode=$?', 'logdata': ''}
{'ClassMessage': 'if [ $hadoop_shell_errorcode -ne 0 ]', 'logdata': ''}
{'ClassMessage': 'then', 'logdata': ''}
{'ClassMessage': '  exit $hadoop_shell_errorcode', 'logdata': ''}
{'ClassMessage': 'fi', 'logdata': ''}
{'ClassMessage': 'ln -sf "/data3/hadoop/yarn/local/filecache/2693/univocity-parsers-1.5.1.jar" "univocity-parsers-1.5.1.jar"', 'logdata': ''}
{'ClassMessage': 'hadoop_shell_errorcode=$?', 'logdata': ''}
{'ClassMessage': 'if [ $hadoop_shell_errorcode -ne 0 ]', 'logdata': ''}
{'ClassMessage': 'then', 'logd

In [174]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
#filename = "application2.log"
#filename = "2020_02_29-application_1580556634479_40162.log"
import csv
outcsv = open("testoutput6.csv", "w")
filename = "testlog.txt"

#outF = open("testoutput", "w")
lg = LineGrokker('%{LOGLEVEL:level}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{JAVALOGMESSAGE:ClassMessage}%{GREEDYDATA:logdata}', pr)

#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))

#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0)
#java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))
pline=[]
cline={}

with open(filename, 'r') as filehandle:
    for line in filehandle:
        #print(lg.grok(line))
        cline=lg.grok(line)
        if (cline is not None) and len(pline)==0:
#            print(cline)
#            outF.write(str(cline))
#            outF.write("\n")
            #pline=[]
            print(1)
            if cline['level']=='ERROR':
                pline.append(cline)
            else:
                for i in cline.keys():
                    outcsv.write("%s," % (cline[i]))
                outcsv.write("\n")
#                outF.write(str(cline))
#                outF.write("\n")
            continue
#        elif (cline is None) and len(pline)!=0:
#            cline=lg2.grok(line)
#            if cline is not None:
                #print(cline)
                #print(2)
#                pline.append(cline)
#                continue
#            cline=lg3.grok(line)
#            if cline is not None:
                #print(cline)
                #print(3)
#                pline.append(cline)                
#            cline=lg4.grok(line)
#            if cline is not None:
#                print(cline)
#                print(4)
#                pline.append(cline)
        elif (cline is not None) and len(pline)!=0:
            cline=lg4.grok(line)
            if cline is not None:
                print(cline)
#                print(4)
                pline.append(cline)        
            for indx in pline:
                for i in indx.keys():
                    outcsv.write("%s," % (indx[i]))
            outcsv.write("\n")
            for i in cline.keys():
                outcsv.write("%s," % (cline[i]))
            outcsv.write("\n")
#            outF.write(str(pline))
#            outF.write("\n")
#            outF.write(str(cline))
#            outF.write("\n")
            pline=[]
#        print(pline)
        
#outF.close()
outcsv.close() 

1
1
{'ClassMessage': '\tat com.merck.mcloud.gxp.rdq.qd.transformations.RdqTransformationTemplate$$anonfun$mapRowToObject$1.apply(RdqTransformationTemplate.scala:202)', 'logdata': ''}
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


### Working code for : 20/02/12 13:18:48

In [41]:
from korg import LineGrokker, PatternRepo
import re
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
#filename = "application2.log"
filename = r"C:\Users\sreddy\OneDrive - MerckGroup\New folder\data\raw_logs\application_1568810042014_190518.log"

import csv
outcsv = open("application_1584006084041_0213.csv", "w")
#filename = "testlog.txt"

#outF = open("testoutput", "w")
lg = LineGrokker('%{LOGLEVEL:level}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{GREEDYDATA:logdata}', pr)
pline=[]
cline={}
with open(filename, 'r') as filehandle:
    for line in filehandle:
#------------------------------------------------------------        
        lineFormat = re.search(r'^(\d+/\d+/\d+)',line)
        if lineFormat != None:
            cline=lg.grok(line)
        else:
            lg = LineGrokker('%{LOGLEVEL:level} %{SPACE}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
            cline=lg.grok(line)
#------------------------------------------------------------        
        #cline=lg.grok(line)
        if (cline is not None) and len(pline)==0:         
            if cline['level']=='ERROR':
                pline.append(cline)
            elif cline['level']=='er':
                cline=lg4.grok(line)
                pline.append(cline)
            else:
                for i in cline.keys():
                    outcsv.write("%s," % (cline[i]))
                outcsv.write("\n")
        elif (cline is not None) and len(pline)!=0:         
            if (cline['level']=='er'):
                cline=lg4.grok(line)
                pline.append(cline)
            elif (cline['level']=='INFO'):
                for indx in pline:
                    for i in indx.keys():
                        outcsv.write("%s " % (indx[i]))
                outcsv.write("\n")
                for i in cline.keys():
                    outcsv.write("%s," % (cline[i]))
                outcsv.write("\n")
                pline=[]                
#            else:
#                print(cline)
#                for i in cline.keys():
#                    outcsv.write("%s," % (cline[i]))
#                outcsv.write("2\n")
        elif (cline is None) and len(pline)!=0:         
            cline=lg4.grok(line)
            pline.append(cline)
outcsv.close()            
            

### Working code for :2020-01-07 13:06:03,622 [Driver]

In [3]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
#filename = "application2.log"
#filename = r"C:\project\uc7\Issues\application_1576136188037_14643.log"
filename = r"C:\Users\sreddy\OneDrive - MerckGroup\New folder\data\raw_logs\1application_1568810042014_190726.log"

import csv
outcsv = open("1application_1568810042014_190726.csv", "w")
#filename = "testlog.txt"

#outF = open("testoutput", "w")
lg = LineGrokker('%{LOGLEVEL:level} %{SPACE}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{GREEDYDATA:logdata}', pr)
pline=[]
cline={}
with open(filename, 'r') as filehandle:
    for line in filehandle:
        cline=lg.grok(line)
        if (cline is not None) and len(pline)==0:         
            if cline['level']=='ERROR':
                pline.append(cline)
            elif cline['level']=='er':
                cline=lg4.grok(line)
                pline.append(cline)
            else:
                for i in cline.keys():
                    outcsv.write("%s," % (cline[i]))
                outcsv.write("\n")
        elif (cline is not None) and len(pline)!=0:         
            if (cline['level']=='er'):
                cline=lg4.grok(line)
                pline.append(cline)
            elif (cline['level']=='INFO'):
                for indx in pline:
                    for i in indx.keys():
                        outcsv.write("%s " % (indx[i]))
                outcsv.write("\n")
                for i in cline.keys():
                    outcsv.write("%s," % (cline[i]))
                outcsv.write("\n")
                pline=[]                
#            else:
#                print(cline)
#                for i in cline.keys():
#                    outcsv.write("%s," % (cline[i]))
#                outcsv.write("2\n")
        elif (cline is None) and len(pline)!=0:         
            cline=lg4.grok(line)
            pline.append(cline)
outcsv.close()

In [4]:
#'%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}'
lg = LineGrokker('%{LOGLEVEL:level} %{SPACE}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#print(lg.grok('''2020-03-12 20:09:17,951 [Executor task launch worker for task 0] ERROR org.apache.spark.executor.Executor  - Exception in task 0.0 in stage 0.0 (TID 0) java.lang.RuntimeException: java.lang.NullPointerException: hive.llap.daemon.service.hosts must be defined'''))
#20/02/27 04:09:10 ERROR ApplicationMaster: User class threw exception: com.merck.mcloud.gxp.pipelinecore.CustomExceptions.SqoopException: Sqoop import unsuccessful. Abort!
print(lg.grok('''Caused by: java.lang.NumberFormatException: For input string: ""'''))
#2020-01-07 13:06:03,146 [Driver] INFO  org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef  - Registered StateStoreCoordinator endpoint

None


In [9]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
#filename = "application2.log"
#filename = r"C:\project\uc7\Issues\application_1576136188037_14643.log"
filename = r"C:\Users\sreddy\OneDrive - MerckGroup\New folder\log_files\1application_1568810042014_190734.log"


import csv
outcsv = open("1application_1568810042014_190734.csv", "w")
#filename = "testlog.txt"

#outF = open("testoutput", "w")
lg = LineGrokker('%{LOGLEVEL:level} %{SPACE}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{GREEDYDATA:logdata}', pr)
pline=[]
cline={}
with open(filename, 'r') as filehandle:
    for line in filehandle:
        cline=lg.grok(line)
        if (cline is not None) and len(pline)==0:         
            if cline['level']=='ERROR':
                pline.append(cline)
            elif cline['level']=='er':
                cline=lg4.grok(line)
                pline.append(cline)
                pass
            else:
                for i in cline.keys():
                    outcsv.write("%s|" % (cline[i]))
                outcsv.write("\n")
        elif (cline is not None) and len(pline)!=0:         
            if (cline['level']=='er'):
                cline=lg4.grok(line)
                pline.append(cline)
                pass
            elif (cline['level']=='INFO'):
                for indx in pline:
                    for i in indx.keys():
                        outcsv.write("%s " % (indx[i]))
                outcsv.write("\n")
                for i in cline.keys():
                    outcsv.write("%s|" % (cline[i]))
                outcsv.write("\n")
                pline=[]                
#            else:
#                print(cline)
#                for i in cline.keys():
#                    outcsv.write("%s," % (cline[i]))
#                outcsv.write("2\n")
        elif (cline is None) and len(pline)!=0:         
            cline=lg4.grok(line)
            pline.append(cline)
outcsv.close()

In [ ]:
fileReadFormat=

In [6]:
from korg import LineGrokker, PatternRepo
pr = PatternRepo()  # use the std. logstash grok patterns
#filename = "application1.log"
#filename = "application2.log"
filename = r"C:\Users\sreddy\OneDrive - MerckGroup\New folder\data\raw_logs\application_1568810042014_190518.log"

import csv
outcsv = open("application_1584006084041_0213.csv", "w")
#filename = "testlog.txt"

#outF = open("testoutput", "w")
lg = LineGrokker('%{LOGLEVEL:level}%{JAVALOGMESSAGE:JavaMessage}%{GREEDYDATA:logdata}', pr)
#lg = LineGrokker('%{TIMESTAMP_ISO8601:timestmp} %{SYSLOG5424SD:prg} %{LOGLEVEL:level}%{SPACE}%{JAVACLASS:ClassName} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg2 = LineGrokker('%{DATA:ClassName} %{DATA:ClassName2} %{JAVALOGMESSAGE:JavaMessage} %{GREEDYDATA:logdata}', pr)
lg3 = LineGrokker('%{JAVASTACKTRACEPART:JavaStackTrace}%{GREEDYDATA:logdata}', pr)
lg4 = LineGrokker('%{GREEDYDATA:logdata}', pr)
pline=[]
cline={}
#with open(filename, 'r') as filehandle:
#    for line in filehandle:
#        cline=lg.grok(line)

In [7]:
with open(filename, 'r') as filehandle:
    for line in filehandle:
        print(line)

Container: container_e175_1568810042014_190518_01_000003 on awdex01009.aws.merckcloud.com_45454_1581588577175


LogType:directory.info

Log Upload Time:Thu Feb 13 10:09:37 +0000 2020

LogLength:14350

Log Contents:

ls -l:

total 156

lrwxrwxrwx 1 s112380 hadoop    82 Feb 13 10:07 __app__.jar -> /data/hadoop/yarn/local/usercache/s112380/filecache/100/qd_rdq_2.10-1.0.1-RC14.jar

lrwxrwxrwx 1 s112380 hadoop    68 Feb 13 10:07 aws-java-sdk-core-1.10.6.jar -> /data/hadoop/yarn/local/filecache/39803/aws-java-sdk-core-1.10.6.jar

lrwxrwxrwx 1 s112380 hadoop    68 Feb 13 10:07 aws-java-sdk-kms-1.10.6.jar -> /data1/hadoop/yarn/local/filecache/39809/aws-java-sdk-kms-1.10.6.jar

lrwxrwxrwx 1 s112380 hadoop    67 Feb 13 10:07 aws-java-sdk-s3-1.10.6.jar -> /data1/hadoop/yarn/local/filecache/39798/aws-java-sdk-s3-1.10.6.jar

lrwxrwxrwx 1 s112380 hadoop    69 Feb 13 10:07 azure-keyvault-core-0.8.0.jar -> /data/hadoop/yarn/local/filecache/39800/azure-keyvault-core-0.8.0.jar

lrwxrwxrwx 1 s112380 hado

20/02/13 10:06:56 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(s112380); users with modify permissions: Set(s112380)

20/02/13 10:06:56 INFO Utils: Successfully started service 'sparkDriver' on port 39477.

20/02/13 10:06:56 INFO Slf4jLogger: Slf4jLogger started

20/02/13 10:06:57 INFO Remoting: Starting remoting

20/02/13 10:06:57 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriverActorSystem@172.31.104.16:49726]

20/02/13 10:06:57 INFO Utils: Successfully started service 'sparkDriverActorSystem' on port 49726.

20/02/13 10:06:57 INFO SparkEnv: Registering MapOutputTracker

20/02/13 10:06:57 INFO SparkEnv: Registering BlockManagerMaster

20/02/13 10:06:57 INFO DiskBlockManager: Created local directory at /data/hadoop/yarn/local/usercache/s112380/appcache/application_1568810042014_190518/blockmgr-46128332-169c-4973-a0ea-deb0a7ebae17

20/02/13 10:06:57 INFO DiskBlockManager: Created local 

In [39]:
import re
match = re.search(r'^(\d+/\d+/\d+)','11/12/1998 The date is ')
print(match)

<re.Match object; span=(0, 10), match='11/12/1998'>


In [25]:
import datetime
from datetime import date
import re
s = "Jason's birthday is on 1991-09-21 "
match = re.search(r'\d{4}-\d{2}-\d{2}', s)
date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
print(date)

1991-09-21


In [26]:
match.group()

'1991-09-21'

In [27]:
re.split(r'\s+(?=\d{2}(?:\d{2})?-\d{1,2}-\d{1,2}\b)', s)

["Jason's birthday is on 1991-09-21 "]

In [28]:
s.startswith(r'\s+(?=\d{2}(?:\d{2})?-\d{1,2}-\d{1,2}\b))

SyntaxError: EOL while scanning string literal (<ipython-input-28-a09575f1a2d2>, line 1)